# Project Imports

In [1]:
from project_utils import config_paths
config_paths('omri', 'my_env') # Change to your name and your environment folder!
from models import Unet

/home/omri/my_gpu/.env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from project_utils import config_paths
config_paths('omri', 'my_env') # Change to your name and your environment folder!
from project_utils import read_images, mean_iou, rle_encoding, prob_to_rles, predict_results, create_submission_file
from skimage.transform import resize
from skimage.morphology import label
from keras.callbacks import EarlyStopping, ModelCheckpoint
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import time

In [3]:
main_tic = time.time()

# Constants

In [5]:
TRAIN_PATH = '../../stage1_train/'
TEST_PATH = '../../stage1_test/'
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

# Read the data

In [6]:
X_train, Y_train, X_test, sizes_test  = read_images(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS, TRAIN_PATH, TEST_PATH)

Getting and resizing train images and masks ... 


100%|██████████| 650/650 [01:37<00:00,  6.64it/s]

Getting and resizing test images ... 



100%|██████████| 65/65 [00:00<00:00, 74.38it/s]

Done!


# Import/Build your model here:

In [8]:
model = Unet.unet(mean_iou, IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 128, 128, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 16) 448         lambda_1[0][0]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 128, 128, 16) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (

# Train

In [10]:
tic = time.time()
model_name = 'lalala' 
model_name = model_name + '.h5'
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint(model_name, verbose=1, save_best_only=True)
results = model.fit(X_train, Y_train, validation_split=0.1, batch_size=16, epochs=2, 
                    callbacks=[earlystopper, checkpointer])

toc = time.time() - tic
print('Training time...', toc/60)

Train on 585 samples, validate on 65 samples
Epoch 1/2
585/585 [==============================] - 8s 13ms/step - loss: 0.4389 - mean_iou: 0.4206 - val_loss: 0.4387 - val_mean_iou: 0.4237

Epoch 00001: val_loss improved from inf to 0.43865, saving model to lalala.h5
Epoch 2/2
585/585 [==============================] - 7s 11ms/step - loss: 0.3290 - mean_iou: 0.4249 - val_loss: 0.3241 - val_mean_iou: 0.4352

Epoch 00002: val_loss improved from 0.43865 to 0.32408, saving model to lalala.h5
Training time... 0.256594709555308


# Predict

In [11]:
preds_test_upsampled = predict_results(model_name, X_train, X_test, sizes_test)

65/65 [==============================] - 0s 2ms/step


# Create a submission file

In [12]:
filename = 'lalalala'
test_ids = os.listdir(TEST_PATH)
sub = create_submission_file(preds_test_upsampled, test_ids, filename)

In [13]:
sub.head()

,ImageId,EncodedPixels
0,0999dab07b11bc85fb8464fc36c947fbd8b5d6ec498173...,47881 11 48276 12 48395 20 48792 18 48912 24 4...
1,0999dab07b11bc85fb8464fc36c947fbd8b5d6ec498173...,119399 10 119907 24 120420 31 120935 37 121453...
2,0999dab07b11bc85fb8464fc36c947fbd8b5d6ec498173...,3678 14 4193 24 4708 34 5224 50 5742 51 6261 5...
3,0999dab07b11bc85fb8464fc36c947fbd8b5d6ec498173...,19866 14 20383 20 20898 25 21413 31 21927 38 2...
4,0999dab07b11bc85fb8464fc36c947fbd8b5d6ec498173...,652 53 1160 68 1676 73 2194 74 2713 75 3231 78...


In [14]:
print('Total runtime...', (time.time() - main_tic)/60)

Total runtime... 9.302050149440765
